## 问题实战 ---文本抄袭判定

In [ ]:
sklearn的使用、文本分类流程

+ 导入数据
+ 文本预处理
+ 文本表征
+ 划分训练集和测试集
+ 训练分类器
+ 验证性能
+ 参数调整
+ 保存模型和再次使用

问题描述： 在新闻出版业中一个常常的问题就是新闻版权抄袭，所以我们现在为了避免这个事情，需要建立一个模型，判断这个文章是不是由某个新闻出版单位出版的。 在我们这个问题里，我们需要 建立一个模型，该模型接受一个作为文本的输入，然后判断该文本是不是由“新华社”发布的。

In [1]:
import numpy as np
import pandas as pd
import os
import csv
import re
import jieba
import random

In [2]:
database = '../L02languageModel/sqlResult_1558435.csv'
os.path.exists(database)

True

In [3]:
content = pd.read_csv(database, encoding='gb18030')

In [5]:
content = content.fillna('')

In [6]:
content['is_xinhua'] = np.where(content['source'].str.contains('新华'),1,0) #np.where(condition, x, y)满足条件(condition)，输出x，不满足输出y

In [7]:
content[:5]

,id,author,source,content,feature,title,url,is_xinhua
0,89617,,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm,0
1,89616,,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm,0
2,89615,,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm,0
3,89614,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....,1
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...,0


In [8]:
def cut(string):
    return ' '.join(jieba.cut(string))

In [9]:
content['title'] = content['title'].apply(cut)   #help(content.apply)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.728 seconds.
Prefix dict has been built succesfully.


In [10]:
content['title'][:5]

0                  小米 MIUI   9 首批 机型 曝光 ： 共计 15 款
1           骁龙 835 在 Windows   10 上 的 性能 表现 有望 改善
2           一加 手机 5 细节 曝光 ： 3300mAh 、 充 半小时 用 1 天
3    葡 森林 火灾 造成 至少 62 人 死亡   政府 宣布 进入 紧急状态 （ 组图 ）
4         44 岁 女子 约 网友 被 拒 暴雨 中 裸奔   交警 为 其 披衣 相随
Name: title, dtype: object

In [11]:
content = content.sample(n=5000)

In [12]:
xinhuashe_news = content[content['source'].str.contains('新华')]

In [13]:
len(xinhuashe_news), len(content)

(4397, 5000)

In [14]:
X_title = content['title'].values

In [15]:
y = content['is_xinhua'].values

In [16]:
X_title[:10]

array(['（ 国际 ） （ 2 ） 第 70 届 戛纳 电影节 主 竞赛 单元 入围 影片 揭晓',
       '（ 一带 一路 · 好 味道 ） （ 1 ） 张骞 故里 的 汉中 面皮',
       '（ 关注 春耕 备耕 ） 田野 上演 农机 总动员 ： 从 “ 谁 来 种地 ” 到 “ 机器 种田 ”',
       '（ 经济 ） 四川 赔付 首单 “ 农地 流转 履约 保证 保险 ”',
       '[ 15 ] （ 外代 二线 ） 动物园 里 的 新 成员',
       '（ 国际 ） 伊拉克 军方 空袭 炸死 超过 １ ５ ０ 名 恐怖分子',
       '（ 社会 ） 调查 显示 美国 多数 州 存在 童婚 情况',
       '霍金 警告 人类 不要 尝试 登月 ？ 中文 媒体 又 丢人 了',
       '【 微 特稿 · 社会 与 生活 】 澳史 上 最大 单笔 善款 \u3000 矿业 大亨 豪捐 ４ 亿 \u3000',
       '（ 全运会 ） （ 2 ） 花样游泳 — — 自由组合 预赛 赛况'], dtype=object)

In [18]:
y[:10]

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=30000)    #only consider the top max_features ordered by term frequency across the corpus
X_title = vectorizer.fit_transform(X_title)

In [20]:
X_title.shape

(5000, 10736)

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(X_title, y, test_size=0.2, random_state=0)
clf = GaussianNB()
clf.fit(X_train.toarray(), y_train)

GaussianNB(priors=None)

In [29]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
def get_performance(clf, X_, y_):
    y_hat = clf.predict(X_.toarray())
    print('percision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))
    print('confusion matrix: \n{}'.format(confusion_matrix(y_, y_hat, labels=[0, 1])))

In [31]:
get_performance(clf,X_train,y_train)

percision is: 1.0
recall is: 0.998006833712984
roc_auc is: 0.9990034168564921
confusion matrix: 
[[ 488    0]
 [   7 3505]]


In [32]:
get_performance(clf,X_test,y_test)

percision is: 0.9363128491620112
recall is: 0.9468926553672317
roc_auc is: 0.7256202407270941
confusion matrix: 
[[ 58  57]
 [ 47 838]]


In [56]:
y_hat_test = clf.predict(X_test.toarray()[:1000])

In [57]:
y_hat_test

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [58]:
candidate_indices = []
for index, (y,yhat) in enumerate(zip(y_test,y_hat_test)):
    if y == 0 and yhat == 1:
        candidate_indices.append(test_indices[index])

In [59]:
candidate_indices

[3779,
 1591,
 1029,
 742,
 3983,
 2252,
 2132,
 3786,
 2798,
 1531,
 2163,
 971,
 3255,
 590,
 480,
 1360,
 4371,
 3766,
 4211,
 1705,
 4734,
 1536,
 2922]

In [60]:
content.iloc[2922]['content']

'新华社照片，布达佩斯，2017年4月15日\\n（国际）（2）布达佩斯动物园母猩猩利赛尔迎来40岁生日\\n\u3000\u30004月15日，母猩猩利赛尔在吃生日蛋糕。\\n匈牙利布达佩斯动物园最年长的母猩猩利赛尔今年40岁，她于1977年4月15日生于德国法兰克福动物园。\\n新华社发（德莫特尔·乔鲍摄）'

In [61]:
content.iloc[2922]['source']

'新华社'

In [62]:
import pickle

In [51]:
with open('clf', 'wb') as picklefile:  
    pickle.dump(clf,picklefile)

In [52]:
with open('clf', 'rb') as training_model:  
    model = pickle.load(training_model)

In [53]:
y_hat_test = model.predict(X_test.toarray()[:1000])
y_hat_test

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [54]:
candidate_indices2 = []
for index, (y,yhat) in enumerate(zip(y_test,y_hat_test)):
    if y == 0 and yhat == 1:
        candidate_indices2.append(test_indices[index])

In [55]:
candidate_indices2

[3779,
 1591,
 1029,
 742,
 3983,
 2252,
 2132,
 3786,
 2798,
 1531,
 2163,
 971,
 3255,
 590,
 480,
 1360,
 4371,
 3766,
 4211,
 1705,
 4734,
 1536,
 2922]